In [ ]:
import cohere
import pinecone
import pandas as pd
import json
from dotenv import load_dotenv

# # === CONFIG ===
# load_dotenv()
# COHERE_API_KEY = os.getenv("COHERE_API_KEY")
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# INDEX_NAME = "sparkathon-walmart"
# EMBED_MODEL = "embed-english-v3.0"  # or "embed-multilingual-v3.0"

# # === INIT ===
# co = cohere.Client(COHERE_API_KEY)

# pinecone.init(api_key=PINECONE_API_KEY)
# if INDEX_NAME not in pinecone.list_indexes():
#     pinecone.create_index(INDEX_NAME, dimension=1024, metric="cosine")  # 1024D for v3.0
# index = pinecone.Index(INDEX_NAME)

# === LOAD DATA ===
df = pd.read_csv("walmart-products.csv")

# === EMBED + UPSERT LOOP ===
for _, row in df.iterrows():
    try:
        # Flatten useful fields
        specs = json.loads(row['specifications']) if pd.notna(row['specifications']) else []
        spec_text = " | ".join([f"{s['name']}: {s['value']}" for s in specs])

        review_data = json.loads(row['customer_reviews']) if pd.notna(row['customer_reviews']) else []
        review_text = " | ".join([f"{r['name']}: {r['review']}" for r in review_data])

        text = f"""
        Product SKU: {row['sku']}
        Brand Info: {spec_text}
        Categories: {row['categories']}
        Price: {row['final_price']} {row['currency']}
        Reviews: {review_text}
        """
        print(text)
        break

        # # === Get Embedding from Cohere ===
        # response = co.embed(
        #     texts=[text],
        #     model=EMBED_MODEL,
        #     input_type="search_document"  # use "search_query" when embedding queries
        # )
        # vector = response.embeddings[0]

        # # === Upsert to Pinecone ===
        # index.upsert([{
        #     "id": str(row["sku"]),
        #     "values": vector,
        #     "metadata": {
        #         "url": row["url"],
        #         "price": row["final_price"],
        #         "brand": [s["value"] for s in specs if s["name"].lower() == "brand"][0] if specs else None,
        #         "categories": row["categories"]
        #     }
        # }])

    except Exception as e:
        print(f"Failed on row {row['sku']}: {e}")




        Product SKU: 173530386
        Title/Description: Laura Mercier Caviar Stick Eye Color Sugar Frost 1.64g/0.05oz Caviar Stick Eye Shadow glides seamlessly onto lids. These eyeshadows are crease transfer and smudge resistant for long lasting even color payoff in a variety of shades and finishes. This easy-to-use versatile cream eye shadow stick delivers effortless application intense buildable color and up to 12-hour long lasting wear. Pick any pigment-rich eye shadow shade in creamy shimmer matte or chrome finish. Caviar Stick Eye Color glides seamlessly onto the lids remaining crease and transfer resistant. The luxe multi-use formulation allows you to easily smudge blend line fill or define eyes so you can effortlessly create any makeup look. An array of intense shades deliver endless options for alluring eyes that stand out both day and night. Dont be afraid to be playful! Caviar Stick Eye Shadow layers easily over or under other eye shadows including powder.
        Brand In

In [17]:
import pandas as pd
df = pd.read_csv("walmart-products.csv")

In [18]:
df.head()

,timestamp,url,final_price,sku,currency,gtin,specifications,image_urls,top_reviews,rating_stars,...,sizes,colors,seller,other_attributes,customer_reviews,ingredients,initial_price,discount,ingredients_full,categories
0,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Laura-Mercier-Cavia...,22.90,173530386,USD,7.361501e+11,"[{""name"":""Brand"",""value"":""Laura Mercier""},{""na...","[""https://i5.walmartimages.com/seo/Laura-Merci...","{""negative"":{},""positive"":{}}","{""five_stars"":2,""four_stars"":3,""two_stars"":1}",...,[],"[""Sugar Frost"",""Tuxedo""]",Wal███t.c███,"[{""name"":""Instructions"",""value"":""Apply directl...","[{""name"":""Jac███"",""rating"":5,""review"":""My only...","Cyclopentasiloxane, trimethylsiloxysilicate, s...",NaN,NaN,"[{""type"":""Ingredients"",""values"":""Cyclopentasil...","[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""..."
1,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Exultantex-Grey-Bla...,47.88,430528189,USD,7.710779e+11,"[{""name"":""Brand"",""value"":""Exultantex""},{""name""...","[""https://i5.walmartimages.com/seo/Exultantex-...","{""negative"":{""rating"":1,""review"":""Color not ac...","{""five_stars"":47,""four_stars"":4,""one_star"":4,""...",...,"[""50\"" x 54\"""",""50\"" x 63\"""",""50\"" x 84\"""",""50...","[""Black"",""Blue"",""Green"",""Gray"",""Natural(Ivory)...",Exu███nte███ome███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Dana"",""rating"":5,""review"":""I love th...",NaN,70.80,$22.92,NaN,"[""Home"",""Decor"",""Curtains & Window Treatments""..."
2,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Jessica-London-Wome...,33.24,6013308220,USD,4.651920e+11,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...","[""https://i5.walmartimages.com/seo/Jessica-Lon...","{""negative"":{},""positive"":{}}","{""five_stars"":2}",...,"[""S"",""M"",""L"",""1X"",""2X"",""3X""]","[""Aqua Sea"",""Dark Olive Green"",""Dark Sapphire""...",Ful███aut███ran███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Bev███y F███"",""rating"":5,""review"":""V...",NaN,37.99,$4.75,NaN,"[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."
3,2024-08-24 00:00:00.000,https://www.walmart.com/ip/100-Cotton-King-Per...,49.99,161657830,USD,8.407081e+11,"[{""name"":""Brand"",""value"":""Simply Put""},{""name""...","[""https://i5.walmartimages.com/asr/d88fe658-b9...","{""negative"":{""rating"":1,""review"":""[This review...","{""five_stars"":78,""four_stars"":17,""one_star"":2,...",...,"[""Queen"",""King""]","[""Beige"",""Blue"",""Gray"",""Spa Blue""]",IC ███bal███c,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Jes███oge███"",""rating"":5,""review"":""[...",NaN,71.99,$22.00,NaN,"[""Home"",""Bedding"",""Duvet Covers"",""King Duvet C..."
4,2024-08-25 00:00:00.000,https://www.walmart.com/ip/Disney-Boys-Graphic...,12.99,5397071399,USD,4.600086e+11,"[{""name"":""Country of Origin - Textiles"",""value...","[""https://i5.walmartimages.com/seo/Disney-Boys...","{""negative"":{""rating"":1,""review"":""This size sh...","{""five_stars"":12,""one_star"":1,""three_stars"":1,...",...,"[""2T"",""3T"",""3T-4T"",""3-4 Years"",""4-5 Years"",""4-...","[""1#Yellow Micky"",""2#Pink Minnie"",""3#Blue Dona...",Pat███,NaN,"[{""name"":""Mary"",""rating"":5,""review"":""Love thes...",NaN,29.00,$16.01,NaN,"[""Clothing"",""Kids Clothing"",""Boys Clothing"",""B..."


In [44]:
import cohere
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import pandas as pd
import json
from dotenv import load_dotenv
import os
from tqdm import tqdm

# === CONFIG ===
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "sparkathon-walmart"
EMBED_MODEL = "embed-v4.0"  # or "embed-multilingual-v3.0"

# === INIT ===
co = cohere.ClientV2(COHERE_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()  # Ensure connection is established
index = pc.Index(INDEX_NAME)

# === LOAD DATA ===
df = pd.read_csv("walmart-products.csv")

# === EMBED + UPSERT LOOP ===
for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        specs = json.loads(row['specifications']) if pd.notna(row['specifications']) else []
        spec_text = " | ".join([f"{s['name']}: {s['value']}" for s in specs])

        review_data = json.loads(row['customer_reviews']) if pd.notna(row['customer_reviews']) else []
        review_text = " | ".join([f"{r.get('name', '')}: {r.get('review', '')}" for r in review_data])

        other_attrs = row['other_attributes'] if pd.notna(row['other_attributes']) else ""
        ingredients = row['ingredients'] if pd.notna(row['ingredients']) else ""
        sizes = row['sizes'] if pd.notna(row['sizes']) else ""
        colors = row['colors'] if pd.notna(row['colors']) else ""
        top_reviews = row['top_reviews'] if pd.notna(row['top_reviews']) else ""
        rating = row['rating'] if pd.notna(row['rating']) else ""
        main_image = row['main_image'] if pd.notna(row['main_image']) else ""
        description = row['description'] if pd.notna(row['description']) else ""
        category_name = row['category_name'] if pd.notna(row['category_name']) else ""
        currency = row['currency'] if pd.notna(row['currency']) else ""

        text = f"""
        Product SKU: {row['sku']}
        Title/Description: {description}
        Brand Info: {spec_text}
        Categories: {row['categories']}
        Category Name: {category_name}
        Price: {row['final_price']} {currency}
        Currency: {currency}
        Top Reviews: {top_reviews}
        Ratings (out of 5): {rating}
        Sizes: {sizes}
        Colours: {colors}
        Main Image: {main_image}
        Other Attributes: {other_attrs}
        Customer Reviews: {review_text}
        Ingredients: {ingredients}
        """

        # === Get Embedding from Cohere ===
        response = co.v2.embed(
            texts=[text],
            model=EMBED_MODEL,
            output_dimension=1024,
            input_type="search_document",  # use "search_query" when embedding queries
            embedding_types=["float"]  # specify embedding type
        )
        vector = response.embeddings.float[0]



        # === Upsert to Pinecone ===
        index.upsert([{
            "id": str(row["sku"]),
            "values": vector,
            "metadata": {
                "url": row["url"],
                "price": row["final_price"],
                "brand": [s["value"] for s in specs if s["name"].lower() == "brand"][0] if specs else None,
                "categories": row["categories"]
            }
        }])

    except Exception as e:
        print(f"Failed on row {row['sku']}: {e}")




  4%|▍         | 45/1000 [00:43<12:05,  1.32it/s]

Failed on row 5702523237: list index out of range


  7%|▋         | 73/1000 [03:23<43:10,  2.79s/it]


KeyboardInterrupt: 

In [54]:
response = co.v2.embed(
            texts=["100% ORGANIC COTTON Queen size bedsheet pink color"],
            model=EMBED_MODEL,
            output_dimension=1024,
            input_type="search_document",  # use "search_query" when embedding queries
            embedding_types=["float"]  # specify embedding type
        )
vector = response.embeddings.float[0]

In [51]:
!pip install pyperclip

  Preparing metadata (setup.py) ... done
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11002 sha256=46bc2c7cca30e0865d0080053f9f0df83cc884aefcadd429aabf08dbdc4aa713
  Stored in directory: /Users/ginger/Library/Caches/pip/wheels/cc/ae/36/ee17d1de094fcb61e24106cb329b5103861e819f94bef5e10a
Successfully built pyperclip


In [55]:
import pyperclip 
pyperclip.copy(vector)